In [1]:
import os

import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

import logging
import time
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/boilers_drive/normalized_df.csv'

Mounted at /content/drive


In [3]:
# Specify the file path of your CSV file
#file_path = 'normalized_df.csv'

# the current script directory
# the current working directory
#current_dir = os.getcwd()

# the path to the CSV file in the parent directory
#parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
#file_path = os.path.join(parent_dir, 'normalized_df.csv')

# Read the CSV file
data = pd.read_csv(file_path)

print("Imported data:")
print(data)

Imported data:
                        time    blr_mod_lvl  absorption  insulation  t_r_set  \
0        2022-10-01 00:00:00   0.000000e+00    0.503910   -7.457292     15.0   
1        2022-10-01 00:01:00   0.000000e+00    0.503910   -7.455208     15.0   
2        2022-10-01 00:02:00   0.000000e+00    0.518558   -7.487500     15.0   
3        2022-10-01 00:03:00   0.000000e+00    0.616207   -7.426042     15.0   
4        2022-10-01 00:04:00   0.000000e+00    0.699210   -7.425000     15.0   
...                      ...            ...         ...         ...      ...   
7331983  2023-04-30 23:55:00  2.871866e-119   -0.296880   -1.604167     17.0   
7331984  2023-04-30 23:56:00  1.914578e-119   -0.296880   -1.614583     17.0   
7331985  2023-04-30 23:57:00  1.276385e-119   -0.296880   -1.572917     17.0   
7331986  2023-04-30 23:58:00  8.509234e-120   -0.296880   -1.511458     17.0   
7331987  2023-04-30 23:59:00  5.672823e-120   -0.296880   -1.432203     17.0   

             t_out house

In [4]:
# Different houses in data
houses = data['house_id'].unique()
print("Different houses in data:")
print(houses)
print("Number of different houses:")
print(len(houses))

Different houses in data:
['home2' 'home9' 'home13' 'home14' 'home34' 'home46' 'home55' 'home67'
 'home86' 'home93' 'home101' 'home106' 'home110' 'home43' 'home63'
 'home53' 'home79' 'home90' 'home95' 'home5' 'home17' 'home47' 'home51'
 'home65' 'home77' 'home89' 'home111' 'home114']
Number of different houses:
28


In [5]:
house_datasets = {}
for house in houses:
    house_datasets[house] = data[data['house_id'] == house]

#print(house_datasets)

**Thoughts for the next steps of the process**

In [6]:
# 1 transformer will get the data for blr_mod_lvl
# and the predictions for the next day of the other 4 categories
# and predict the blr_mod_lvl
# (randomly, about 20% of houses for test and the remaining: about 20% validation and the rest training)
# 
# for the transformer that predicts one of the other categories
# for the data (of each house) : 20% test, 20% validation and the remaining test
#

**Δημιουργία του encoder decoder transformer**

In [7]:
# positional encoding layer
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate([np.sin(angle_rads), np.cos(angle_rads)], axis=-1)

  return tf.cast(pos_encoding, dtype=tf.float32)


# positional embedding layer
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x

In [8]:
# Attention layers
# These are all identical except for how the attention is configured

# base attention layer
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()


# cross attention layer
# (at the center of the Transformer is the cross-attention layer, it connects the encoder and decoder)
class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(query=x, key=context, value=context, return_attention_scores=True)
    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x


# global self attention layer
# (it is responsible for processing the context sequence, and propagating information along its length)
class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(query=x, value=x, key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x


# causal self attention layer
# (it does a similar job as the global self attention layer, for the output sequence)
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(query=x, value=x, key=x, use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [9]:
# feed forward network
# (the transformer also includes this point-wise feed-forward network in both the encoder and decoder)
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x

In [10]:
# encoder layer
# (the encoder contains a stack of N encoder layers. Where each EncoderLayer contains
#   a GlobalSelfAttention and FeedForward layer)
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(num_heads=num_heads, key_dim=d_model, dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

In [11]:
# The Encoder
class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size, d_model=d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model, num_heads=num_heads, dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)


  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.
    # Add dropout.
    x = self.dropout(x)
    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.

In [12]:
# decoder layer
# (the decoder's stack is slightly more complex, with each DecoderLayer containing
#   a CausalSelfAttention, a CrossAttention, and a FeedForward layer)
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, *, d_model, num_heads, dff, dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads, key_dim=d_model, dropout=dropout_rate)

    self.cross_attention = CrossAttention(
        num_heads=num_heads, key_dim=d_model, dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)
    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores
    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

In [13]:
# The Decoder
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size, dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size, d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads, dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None


  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)
    x = self.dropout(x)
    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores
    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

In [14]:
# The Transformer
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()

    self.encoder = Encoder(num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size, dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size, dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)


  #def call(self, inputs):
  #  # To use a Keras model with `.fit` you must pass all your inputs in the first argument.
  #  context, x  = inputs
  #  context = self.encoder(context)  # (batch_size, context_len, d_model)
  #  x = self.decoder(x, context)  # (batch_size, target_len, d_model)
  #  # Final linear layer output.
  #  logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)
  #  try:
  #    # Drop the keras mask, so it doesn't scale the losses/metrics.
  #    # b/250038731
  #    del logits._keras_mask
  #  except AttributeError:
  #    pass
  #  # Return the final output and the attention weights.
  #  return logits

  def call(self, inputs):  # Expecting only one input
    x = inputs
    context = self.encoder(x)  # Use encoder directly
    # Assuming 'context' is used for decoding
    x = self.decoder(x, context)
    logits = self.final_layer(x)
    try:
      del logits._keras_mask
    except AttributeError:
      pass
    return logits

**Preparing the data**

In [15]:
# Preparing one instance of transformer for testing

# Define input sequence length and output sequence length
input_seq_length = 24 * 60  # 24 hours * 60 minutes
output_seq_length = 24 * 60  # 24 hours * 60 minutes

# Define the model
model = Transformer(num_layers = 4, d_model = 128, dff = 512, num_heads = 8,
    input_vocab_size=input_seq_length, target_vocab_size=output_seq_length, dropout_rate=0.1)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
metrics = [tf.keras.metrics.MeanAbsoluteError()]
model.compile(optimizer=optimizer, loss='mse', metrics=metrics)

# Train the model


In [16]:
# the categories for prediction
final_category = 'blr_mod_lvl'
prediction_categories = ['absorption', 'insulation', 't_r_set', 't_out']


In [17]:
# Filter data for the 'absorption' category
category = 'absorption'
filtered_data = data[['house_id', 'time', category]].copy()

# We have 1440 minutes per day
minutes_per_day = 1440
batch_size = 8 # Reduce the batch size, in case it helps !

# Function to create input-target pairs for a single house
def create_pairs(house_data, category, minutes_per_day=1440):
    house_data = house_data.sort_values(by='time')
    data = house_data[category].values
    days = len(data) // minutes_per_day
    data = data[:days * minutes_per_day].reshape((days, minutes_per_day))
    input_data = data[:-1]
    target_data = data[1:]
    return input_data, target_data

# Initialize lists to store input and target pairs
input_data_list, target_data_list = [], []

for house_id in filtered_data['house_id'].unique():
    house_data = filtered_data[filtered_data['house_id'] == house_id]
    input_data, target_data = create_pairs(house_data, category)
    input_data_list.append(input_data)
    target_data_list.append(target_data)

# Combine all houses' data
input_data = np.concatenate(input_data_list, axis=0)
target_data = np.concatenate(target_data_list, axis=0)

# Convert to TensorFlow datasets
dataset = tf.data.Dataset.from_tensor_slices((input_data, target_data))
dataset = dataset.cache().shuffle(1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Define the Transformer model
#model = Transformer(num_layers=4, d_model=128, num_heads=8, dff=512, input_vocab_size=minutes_per_day,
#                    target_vocab_size=minutes_per_day, dropout_rate=0.1) # minutes_per_day = 1440
model = Transformer(num_layers=4, d_model=64, num_heads=4, dff=512, input_vocab_size=minutes_per_day,
                    target_vocab_size=minutes_per_day, dropout_rate=0.1) # minutes_per_day = 1440

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])


In [18]:
# Train the model
model.fit(dataset, epochs=10)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sub defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/dist-packages/